# Model

> Scripts to train the models

In [1]:
#| default_exp model

In [2]:
#| export
#| hide
from tensorflow.keras.optimizers import Adam, SGD
import tsgm.models

2024-06-19 09:32:59.680221: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-19 09:33:04.085476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 09:33:06.847173: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15786 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:18:00.0, compute capability: 8.6
2024-06-19 09:33:06.848138: I tensorflow/core/common_runtime/gpu/gpu_device.cc:15

In [3]:
#| export
def get_model(params):
    model_name = params['model_name']

    if model_name == 'vae_conv5':
        # Accessing model configuration from the zoo using parameters from the dictionary
        architecture = tsgm.models.zoo[model_name](
            seq_len=params['seq_len'], 
            feat_dim=params['feature_dim'], 
            latent_dim=params['latent_dim']
        )

        # Extracting encoder and decoder from the architecture
        encoder, decoder = architecture.encoder, architecture.decoder

        # Build the VAE
        vae = tsgm.models.cvae.BetaVAE(encoder, decoder)
        return vae

    elif model_name == 'timeGAN':
        model = tsgm.models.timeGAN.TimeGAN(
            seq_len=params['seq_len'],
            module="gru",
            hidden_dim=24,
            n_features=params['feature_dim'],
            n_layers=3,
            batch_size=256,
            gamma=1.0,
        )
        # .compile() sets all optimizers to Adam by default
        model.compile()
        return model

    else:
        raise ValueError(f"Unsupported model_name: {model_name}")

In [4]:
#| export
def get_optimizer(optimizer_config):
    name = optimizer_config['name'].lower()
    if name == 'adam':
        return Adam(learning_rate=optimizer_config.get('learning_rate', 0.001))
    elif name == 'sgd':
        return SGD(learning_rate=optimizer_config.get('learning_rate', 0.01))
    # Add additional optimizers as needed
    raise ValueError("Unsupported optimizer: {}".format(optimizer_config['name']))

In [5]:
#| hide
import nbdev; nbdev.nbdev_export()